[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hw-oh/wandb_e2e_demo/blob/main/models/image_detection/image_detection.ipynb)

In [ ]:
!pip install -q wandb ultralytics wandb-workspaces

In [ ]:
import wandb
import os
from google.colab import userdata

WANDB_API_KEY = userdata.get("WANDB_API_KEY")
WANDB_ENTITY = userdata.get("WANDB_ENTITY")
WANDB_PROJECT = userdata.get("WANDB_PROJECT")
WANDB_REGISTRY_NAME = userdata.get("WANDB_REGISTRY_NAME")

os.environ["WANDB_API_KEY"] = WANDB_API_KEY

wandb.login(key=WANDB_API_KEY)
print(f"Entity: {WANDB_ENTITY}")
print(f"Project: {WANDB_PROJECT}")

# Unified Vision Demo — YOLOv8-seg로 Classification + Detection + Segmentation

## 개요

이 노트북은 **COCO128-seg** 데이터셋과 **YOLOv8n-seg** 모델을 사용하여
하나의 모델로 **Classification(분류)**, **Detection(탐지)**, **Segmentation(분할)** 을
모두 수행하면서 W&B의 핵심 기능을 전부 체험합니다.

### 왜 YOLOv8-seg인가?

YOLOv8-seg는 한 번의 추론으로 세 가지 결과를 동시에 제공합니다:
- **Classification**: 검출된 객체의 클래스 분류
- **Detection**: 바운딩 박스 좌표 + 클래스 + 신뢰도
- **Segmentation**: 인스턴스별 픽셀 마스크

## 다루는 W&B 기능

| 기능 | 설명 |
|------|------|
| **Experiment Tracking** | 학습 메트릭 실시간 추적 (`wandb.init`, `wandb.log`, `wandb.config`) |
| **Media Logging** | BBox + Mask 시각화 (`wandb.Image`) |
| **Tables** | 데이터셋 미리보기 및 예측 결과 비교 (`wandb.Table`) |
| **Artifacts** | 데이터셋/모델 버저닝 및 계보(lineage) 추적 |
| **Model Registry** | 모델 등록 및 alias 관리 (staging → production) |
| **Sweeps** | 베이지안 하이퍼파라미터 최적화 + 튜닝 효과 검증 |
| **Reports** | 프로그래밍 방식 실험 리포트 생성 |

## 데이터셋
- **COCO128-seg**: COCO 데이터셋의 소형 서브셋 (128장, 80 classes, 인스턴스 세그멘테이션 라벨 포함)
- Ultralytics 내장 — 자동 다운로드

## 모델
- **YOLOv8n-seg** (nano segmentation) — Ultralytics pretrained on COCO
- Detection + Instance Segmentation 통합 모델
- Colab T4에서 빠른 학습, Streamlit Cloud(CPU)에서 추론 가능

In [ ]:
from ultralytics import YOLO
from ultralytics.data.utils import check_det_dataset
from PIL import Image
import numpy as np
import random
import glob
import yaml
import csv

BASELINE_CONFIG = {
    "model_name": "yolov8n-seg",
    "dataset": "coco128-seg",
    "epochs": 30,
    "imgsz": 640,
    "lr0": 0.01,
    "batch": 16,
    "num_classes": 80,
}

COCO_CLASSES = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
    "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
    "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
    "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
    "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
    "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
    "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
    "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear",
    "hair drier", "toothbrush",
]

CLASS_LABELS = {i: name for i, name in enumerate(COCO_CLASSES)}

print(f"COCO classes: {len(COCO_CLASSES)}개")

## 1. 데이터셋 준비 + Artifact 등록

In [ ]:
data_info = check_det_dataset("coco128-seg.yaml")
DATASET_DIR = data_info["path"]
print(f"데이터셋 경로: {DATASET_DIR}")

train_images = glob.glob(f"{DATASET_DIR}/images/train2017/*.jpg")
print(f"Train 이미지: {len(train_images)}장")

In [ ]:
run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config=BASELINE_CONFIG,
    job_type="data-versioning",
    name="coco128-seg-data-versioning",
)

artifact = wandb.Artifact(
    "coco128-seg",
    type="dataset",
    description="COCO128-seg dataset (128 images, 80 classes, instance segmentation labels)",
    metadata={
        "num_images": len(train_images),
        "num_classes": 80,
        "classes": COCO_CLASSES,
        "format": "YOLO-seg",
        "source": "Ultralytics COCO128-seg",
    },
)
artifact.add_dir(DATASET_DIR)
run.log_artifact(artifact)
print("데이터셋 Artifact 로깅 완료!")

In [ ]:
def parse_yolo_label(label_path, img_w, img_h):
    """YOLO-seg format 라벨에서 BBox를 wandb box_data로 변환한다."""
    box_data = []
    if not os.path.exists(label_path):
        return box_data
    with open(label_path) as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            cls_id = int(parts[0])
            x_c, y_c, w, h = float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])
            box_data.append({
                "position": {"middle": [x_c, y_c], "width": w, "height": h},
                "class_id": cls_id,
                "box_caption": COCO_CLASSES[cls_id] if cls_id < len(COCO_CLASSES) else str(cls_id),
            })
    return box_data


table = wandb.Table(columns=["Image with BBox", "Image", "Num Objects", "Classes"])

sample_images = random.sample(train_images, min(20, len(train_images)))

for img_path in sample_images:
    img = Image.open(img_path)
    img_w, img_h = img.size

    label_path = img_path.replace("/images/", "/labels/").replace(".jpg", ".txt")
    box_data = parse_yolo_label(label_path, img_w, img_h)

    img_with_boxes = wandb.Image(
        img,
        boxes={
            "ground_truth": {
                "box_data": box_data,
                "class_labels": CLASS_LABELS,
            }
        } if box_data else {},
    )

    class_names = list({bd["box_caption"] for bd in box_data})
    table.add_data(img_with_boxes, wandb.Image(img), len(box_data), ", ".join(class_names))

wandb.log({"dataset_preview": table})
wandb.finish()
print("데이터셋 미리보기 테이블 로깅 완료!")

## 2. Baseline 학습 — 기본 하이퍼파라미터

먼저 기본 하이퍼파라미터(`lr0=0.01`, `batch=16`)로 YOLOv8n-seg를 학습합니다.
이 결과를 기준점(baseline)으로 삼아, 뒤에서 Sweep을 통해 얼마나 개선되는지 비교합니다.

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config=BASELINE_CONFIG,
    job_type="training",
    name="baseline-yolov8n-seg",
    tags=["baseline"],
)

data_artifact = run.use_artifact(f"{WANDB_ENTITY}/{WANDB_PROJECT}/coco128-seg:latest")

model = YOLO("yolov8n-seg.pt")

results = model.train(
    data="coco128-seg.yaml",
    epochs=BASELINE_CONFIG["epochs"],
    imgsz=BASELINE_CONFIG["imgsz"],
    lr0=BASELINE_CONFIG["lr0"],
    batch=BASELINE_CONFIG["batch"],
    project="runs/segment",
    name="baseline",
    exist_ok=True,
    verbose=True,
)

csv_path = os.path.join(results.save_dir, "results.csv")
if os.path.exists(csv_path):
    with open(csv_path) as f:
        reader = csv.DictReader(f)
        for row in reader:
            row = {k.strip(): v.strip() for k, v in row.items()}
            wandb.log({
                "epoch": int(row["epoch"]),
                "train/box_loss": float(row["train/box_loss"]),
                "train/seg_loss": float(row["train/seg_loss"]),
                "train/cls_loss": float(row["train/cls_loss"]),
                "train/dfl_loss": float(row["train/dfl_loss"]),
                "val/box_loss": float(row["val/box_loss"]),
                "val/seg_loss": float(row["val/seg_loss"]),
                "val/cls_loss": float(row["val/cls_loss"]),
                "val/dfl_loss": float(row["val/dfl_loss"]),
                "val/mAP50_box": float(row["metrics/mAP50(B)"]),
                "val/mAP50-95_box": float(row["metrics/mAP50-95(B)"]),
                "val/precision_box": float(row["metrics/precision(B)"]),
                "val/recall_box": float(row["metrics/recall(B)"]),
                "val/mAP50_mask": float(row["metrics/mAP50(M)"]),
                "val/mAP50-95_mask": float(row["metrics/mAP50-95(M)"]),
            })

BASELINE_BEST_PT = os.path.join(results.save_dir, "weights/best.pt")
baseline_metrics = results.results_dict
baseline_mAP50_box = baseline_metrics.get("metrics/mAP50(B)", 0)
baseline_mAP50_mask = baseline_metrics.get("metrics/mAP50(M)", 0)

print(f"Baseline 학습 완료!")
print(f"  mAP50 (Box):  {baseline_mAP50_box:.4f}")
print(f"  mAP50 (Mask): {baseline_mAP50_mask:.4f}")

In [ ]:
best_model = YOLO(BASELINE_BEST_PT)
sample_for_viz = random.sample(train_images, min(16, len(train_images)))

det_images = []
seg_images = []
for img_path in sample_for_viz:
    img = Image.open(img_path)
    img_w, img_h = img.size
    preds = best_model(img_path, verbose=False)
    r = preds[0]

    box_data = []
    for box in r.boxes:
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        box_data.append({
            "position": {
                "minX": x1 / img_w, "minY": y1 / img_h,
                "maxX": x2 / img_w, "maxY": y2 / img_h,
            },
            "class_id": int(box.cls),
            "scores": {"confidence": float(box.conf)},
            "box_caption": f"{COCO_CLASSES[int(box.cls)]} {float(box.conf):.2f}",
        })

    det_images.append(wandb.Image(
        img,
        boxes={"predictions": {"box_data": box_data, "class_labels": CLASS_LABELS}} if box_data else {},
    ))

    rendered = Image.fromarray(r.plot()[..., ::-1])
    seg_images.append(wandb.Image(rendered, caption=f"{len(r.boxes)}개 객체"))

wandb.log({"baseline/detection_results": det_images})
wandb.log({"baseline/segmentation_results": seg_images})
print(f"{len(sample_for_viz)}장에 대한 Detection + Segmentation 시각화 로깅 완료!")

In [ ]:
pred_table = wandb.Table(columns=[
    "Detection", "Seg Overlay", "Num Objects", "Detected Classes", "Avg Confidence",
])

eval_images = random.sample(train_images, min(30, len(train_images)))

for img_path in eval_images:
    img = Image.open(img_path)
    img_w, img_h = img.size
    preds = best_model(img_path, verbose=False)
    r = preds[0]

    box_data = []
    detected_classes = set()
    confs = []
    for box in r.boxes:
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        cls_id = int(box.cls)
        conf = float(box.conf)
        box_data.append({
            "position": {
                "minX": x1 / img_w, "minY": y1 / img_h,
                "maxX": x2 / img_w, "maxY": y2 / img_h,
            },
            "class_id": cls_id,
            "scores": {"confidence": conf},
            "box_caption": f"{COCO_CLASSES[cls_id]} {conf:.2f}",
        })
        detected_classes.add(COCO_CLASSES[cls_id])
        confs.append(conf)

    det_img = wandb.Image(
        img,
        boxes={"predictions": {"box_data": box_data, "class_labels": CLASS_LABELS}} if box_data else {},
    )
    seg_overlay = wandb.Image(Image.fromarray(r.plot()[..., ::-1]))
    avg_conf = np.mean(confs) if confs else 0.0

    pred_table.add_data(
        det_img, seg_overlay, len(r.boxes),
        ", ".join(sorted(detected_classes)), f"{avg_conf:.2%}",
    )

wandb.log({"baseline/prediction_table": pred_table})
print(f"예측 결과 테이블 ({len(eval_images)}장) 로깅 완료!")

In [ ]:
baseline_artifact = wandb.Artifact(
    "yolov8n-seg-coco128",
    type="model",
    description="YOLOv8n-seg baseline on COCO128-seg (default hyperparameters)",
    metadata={
        "model_type": "yolo-seg",
        "model_architecture": "yolov8n-seg",
        "dataset": "coco128-seg",
        "num_classes": 80,
        "classes": COCO_CLASSES,
        "framework": "ultralytics",
        "input_size": [3, 640, 640],
        "epochs": BASELINE_CONFIG["epochs"],
        "best_mAP50": baseline_mAP50_box,
        "best_mAP50_mask": baseline_mAP50_mask,
        "sweep_tuned": False,
    },
)
baseline_artifact.add_file(BASELINE_BEST_PT, name="best.pt")
run.log_artifact(baseline_artifact)

run.link_artifact(
    baseline_artifact,
    f"{WANDB_REGISTRY_NAME}/coco128-vision",
    aliases=["baseline"],
)
print("Baseline 모델을 Registry에 'baseline' alias로 등록 완료!")

wandb.finish()

## 3. W&B Sweep — 하이퍼파라미터 최적화

W&B Sweeps의 **베이지안 최적화**를 사용하여 최적의 하이퍼파라미터를 탐색합니다.
Sweep을 여러 번 돌리면서 최적의 조합을 찾아가는 과정을 관찰하세요.

**탐색 파라미터:**
- `lr0`: 초기 학습률 (0.001 ~ 0.02)
- `lrf`: 최종 학습률 비율 (0.01 ~ 0.2)
- `momentum`: SGD 모멘텀 (0.85 ~ 0.98)
- `weight_decay`: 가중치 감쇠 (0.0001 ~ 0.001)
- `warmup_epochs`: 웜업 에포크 수 (1, 2, 3)
- `mosaic`: 모자이크 증강 비율 (0.5 ~ 1.0)

**최적화 목표:** `val/mAP50_box` 최대화

In [ ]:
sweep_config = {
    "method": "bayes",
    "metric": {"name": "val/mAP50_box", "goal": "maximize"},
    "parameters": {
        "lr0": {"min": 0.001, "max": 0.02},
        "lrf": {"min": 0.01, "max": 0.2},
        "momentum": {"min": 0.85, "max": 0.98},
        "weight_decay": {"min": 0.0001, "max": 0.001},
        "warmup_epochs": {"values": [1, 2, 3]},
        "mosaic": {"min": 0.5, "max": 1.0},
    },
}

SWEEP_EPOCHS = 10


def sweep_train():
    run = wandb.init()
    cfg = wandb.config

    sweep_model = YOLO("yolov8n-seg.pt")
    sweep_results = sweep_model.train(
        data="coco128-seg.yaml",
        epochs=SWEEP_EPOCHS,
        imgsz=640,
        lr0=cfg.lr0,
        lrf=cfg.lrf,
        momentum=cfg.momentum,
        weight_decay=cfg.weight_decay,
        warmup_epochs=cfg.warmup_epochs,
        mosaic=cfg.mosaic,
        batch=16,
        project="runs/segment_sweep",
        name=f"sweep_{run.id}",
        exist_ok=True,
        verbose=False,
    )

    metrics = sweep_results.results_dict
    wandb.log({
        "val/mAP50_box": metrics.get("metrics/mAP50(B)", 0),
        "val/mAP50-95_box": metrics.get("metrics/mAP50-95(B)", 0),
        "val/precision_box": metrics.get("metrics/precision(B)", 0),
        "val/recall_box": metrics.get("metrics/recall(B)", 0),
        "val/mAP50_mask": metrics.get("metrics/mAP50(M)", 0),
        "val/mAP50-95_mask": metrics.get("metrics/mAP50-95(M)", 0),
    })
    wandb.finish()


sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT, entity=WANDB_ENTITY)
wandb.agent(sweep_id, function=sweep_train, count=10)
print("Sweep 완료!")

## 4. Sweep 최적 하이퍼파라미터로 Full Training

Sweep에서 찾은 최적의 하이퍼파라미터로 full epoch 학습을 실행합니다.
Baseline과 비교하여 Sweep의 효과를 확인합니다.

In [ ]:
api = wandb.Api()
sweep = api.sweep(f"{WANDB_ENTITY}/{WANDB_PROJECT}/{sweep_id}")
best_run = sweep.best_run()
best_config = best_run.config

print("Sweep 최적 하이퍼파라미터:")
for k, v in best_config.items():
    print(f"  {k}: {v}")
print(f"  Best mAP50 (Box): {best_run.summary.get('val/mAP50_box', 'N/A')}")

In [ ]:
tuned_config = {**BASELINE_CONFIG, **best_config}
tuned_config["sweep_tuned"] = True
tuned_config["sweep_id"] = sweep_id

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config=tuned_config,
    job_type="training",
    name="tuned-yolov8n-seg",
    tags=["sweep-tuned"],
)

tuned_model = YOLO("yolov8n-seg.pt")
tuned_results = tuned_model.train(
    data="coco128-seg.yaml",
    epochs=BASELINE_CONFIG["epochs"],
    imgsz=640,
    lr0=best_config.get("lr0", 0.01),
    lrf=best_config.get("lrf", 0.01),
    momentum=best_config.get("momentum", 0.937),
    weight_decay=best_config.get("weight_decay", 0.0005),
    warmup_epochs=best_config.get("warmup_epochs", 3),
    mosaic=best_config.get("mosaic", 1.0),
    batch=16,
    project="runs/segment",
    name="tuned",
    exist_ok=True,
    verbose=True,
)

csv_path = os.path.join(tuned_results.save_dir, "results.csv")
if os.path.exists(csv_path):
    with open(csv_path) as f:
        reader = csv.DictReader(f)
        for row in reader:
            row = {k.strip(): v.strip() for k, v in row.items()}
            wandb.log({
                "epoch": int(row["epoch"]),
                "train/box_loss": float(row["train/box_loss"]),
                "train/seg_loss": float(row["train/seg_loss"]),
                "train/cls_loss": float(row["train/cls_loss"]),
                "train/dfl_loss": float(row["train/dfl_loss"]),
                "val/box_loss": float(row["val/box_loss"]),
                "val/seg_loss": float(row["val/seg_loss"]),
                "val/cls_loss": float(row["val/cls_loss"]),
                "val/dfl_loss": float(row["val/dfl_loss"]),
                "val/mAP50_box": float(row["metrics/mAP50(B)"]),
                "val/mAP50-95_box": float(row["metrics/mAP50-95(B)"]),
                "val/precision_box": float(row["metrics/precision(B)"]),
                "val/recall_box": float(row["metrics/recall(B)"]),
                "val/mAP50_mask": float(row["metrics/mAP50(M)"]),
                "val/mAP50-95_mask": float(row["metrics/mAP50-95(M)"]),
            })

TUNED_BEST_PT = os.path.join(tuned_results.save_dir, "weights/best.pt")
tuned_metrics = tuned_results.results_dict
tuned_mAP50_box = tuned_metrics.get("metrics/mAP50(B)", 0)
tuned_mAP50_mask = tuned_metrics.get("metrics/mAP50(M)", 0)

print(f"\nTuned 학습 완료!")
print(f"  mAP50 (Box):  {tuned_mAP50_box:.4f}")
print(f"  mAP50 (Mask): {tuned_mAP50_mask:.4f}")

In [ ]:
print("=" * 50)
print("Baseline vs Sweep-Tuned 비교")
print("=" * 50)

box_diff = tuned_mAP50_box - baseline_mAP50_box
mask_diff = tuned_mAP50_mask - baseline_mAP50_mask

print(f"{'Metric':<20} {'Baseline':>10} {'Tuned':>10} {'Diff':>10}")
print("-" * 50)
print(f"{'mAP50 (Box)':<20} {baseline_mAP50_box:>10.4f} {tuned_mAP50_box:>10.4f} {box_diff:>+10.4f}")
print(f"{'mAP50 (Mask)':<20} {baseline_mAP50_mask:>10.4f} {tuned_mAP50_mask:>10.4f} {mask_diff:>+10.4f}")

wandb.log({
    "comparison/baseline_mAP50_box": baseline_mAP50_box,
    "comparison/tuned_mAP50_box": tuned_mAP50_box,
    "comparison/improvement_mAP50_box": box_diff,
    "comparison/baseline_mAP50_mask": baseline_mAP50_mask,
    "comparison/tuned_mAP50_mask": tuned_mAP50_mask,
    "comparison/improvement_mAP50_mask": mask_diff,
})

In [ ]:
final_mAP50 = max(tuned_mAP50_box, baseline_mAP50_box)
final_mAP50_mask = max(tuned_mAP50_mask, baseline_mAP50_mask)
final_pt = TUNED_BEST_PT if tuned_mAP50_box >= baseline_mAP50_box else BASELINE_BEST_PT
is_tuned_better = tuned_mAP50_box >= baseline_mAP50_box

print(f"최종 모델: {'Tuned' if is_tuned_better else 'Baseline'} (mAP50 Box: {final_mAP50:.4f})")

model_artifact = wandb.Artifact(
    "yolov8n-seg-coco128",
    type="model",
    description=f"YOLOv8n-seg {'sweep-tuned' if is_tuned_better else 'baseline'} on COCO128-seg",
    metadata={
        "model_type": "yolo-seg",
        "model_architecture": "yolov8n-seg",
        "dataset": "coco128-seg",
        "num_classes": 80,
        "classes": COCO_CLASSES,
        "framework": "ultralytics",
        "input_size": [3, 640, 640],
        "epochs": BASELINE_CONFIG["epochs"],
        "best_mAP50": final_mAP50,
        "best_mAP50_mask": final_mAP50_mask,
        "sweep_tuned": is_tuned_better,
    },
)
model_artifact.add_file(final_pt, name="best.pt")
run.log_artifact(model_artifact)

run.link_artifact(
    model_artifact,
    f"{WANDB_REGISTRY_NAME}/coco128-vision",
    aliases=["staging"],
)
print("최종 모델을 Registry에 'staging' alias로 등록 완료!")

wandb.finish()

## 5. W&B Report 생성

In [ ]:
import wandb_workspaces.reports.v2 as wr

report = wr.Report(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    title="Unified Vision — YOLOv8n-seg 실험 결과 리포트",
    description="YOLOv8n-seg COCO128 학습 + Sweep 하이퍼파라미터 튜닝 결과",
)

report.blocks = [
    wr.TableOfContents(),

    wr.H1("1. 실험 개요"),
    wr.P(
        "COCO128-seg 데이터셋에 대한 YOLOv8n-seg 통합 비전 모델 실험 결과를 정리합니다. "
        "하나의 모델로 Classification, Detection, Segmentation을 모두 수행합니다. "
        "W&B의 Experiment Tracking, Artifacts, Sweeps, Model Registry, Media Logging 기능을 활용하였습니다."
    ),

    wr.H1("2. Baseline 학습 결과"),
    wr.PanelGrid(
        runsets=[
            wr.Runset(entity=WANDB_ENTITY, project=WANDB_PROJECT)
        ],
        panels=[
            wr.LinePlot(title="Box Loss (Train)", x="epoch", y=["train/box_loss"]),
            wr.LinePlot(title="Seg Loss (Train)", x="epoch", y=["train/seg_loss"]),
            wr.LinePlot(title="mAP@0.5 (Box)", x="epoch", y=["val/mAP50_box"]),
            wr.LinePlot(title="mAP@0.5 (Mask)", x="epoch", y=["val/mAP50_mask"]),
            wr.LinePlot(title="Precision (Box)", x="epoch", y=["val/precision_box"]),
            wr.LinePlot(title="Recall (Box)", x="epoch", y=["val/recall_box"]),
        ],
    ),

    wr.H1("3. Sweep 분석"),
    wr.P("Bayesian 최적화를 통한 하이퍼파라미터 탐색 결과:"),
    wr.PanelGrid(
        runsets=[
            wr.Runset(entity=WANDB_ENTITY, project=WANDB_PROJECT)
        ],
        panels=[
            wr.ParallelCoordinatesPlot(
                columns=[
                    wr.ParallelCoordinatesPlotColumn(metric="c::lr0"),
                    wr.ParallelCoordinatesPlotColumn(metric="c::lrf"),
                    wr.ParallelCoordinatesPlotColumn(metric="c::momentum"),
                    wr.ParallelCoordinatesPlotColumn(metric="c::weight_decay"),
                    wr.ParallelCoordinatesPlotColumn(metric="c::mosaic"),
                    wr.ParallelCoordinatesPlotColumn(metric="val/mAP50_box"),
                ],
            ),
            wr.ScalarChart(title="Best mAP@0.5 (Box)", metric="val/mAP50_box"),
            wr.ScalarChart(title="Best mAP@0.5 (Mask)", metric="val/mAP50_mask"),
            wr.BarPlot(title="mAP@0.5 by Run", metrics=["val/mAP50_box"]),
        ],
    ),

    wr.H1("4. Baseline vs Sweep-Tuned 비교"),
    wr.P(
        f"Baseline mAP50 (Box): {baseline_mAP50_box:.4f} → "
        f"Tuned mAP50 (Box): {tuned_mAP50_box:.4f} "
        f"(차이: {box_diff:+.4f})"
    ),

    wr.H1("5. 다음 단계"),
    wr.P(
        "최적 모델을 Model Registry의 'production' alias로 승격하여 "
        "Automation → GitHub Actions → Streamlit 배포 파이프라인을 트리거합니다."
    ),
]

report.save()
print(f"Report 생성 완료! URL: {report.url}")

## 6. Production 승격 (Automation 트리거)

아래 셀을 실행하면 Model Registry에서 최신 모델을 `production` alias로 승격합니다.

W&B Automation이 설정되어 있으면:
1. `production` alias 추가 이벤트 발생
2. Webhook → GitHub `repository_dispatch` 트리거
3. GitHub Actions가 `deployment.json` 업데이트 → git push
4. Streamlit Cloud 앱이 새 모델로 자동 배포

**참고**: W&B UI에서 수동으로 `production` alias를 추가해도 동일하게 동작합니다.

In [ ]:
api = wandb.Api()
artifact_path = f"{WANDB_ENTITY}/{WANDB_PROJECT}/yolov8n-seg-coco128:latest"

try:
    art = api.artifact(artifact_path)
    art.aliases.append("production")
    art.save()
    print(f"'{artifact_path}'에 'production' alias 추가 완료!")
    print("W&B Automation이 설정되어 있으면 배포 파이프라인이 자동으로 트리거됩니다.")
except Exception as e:
    print(f"Production 승격 실패: {e}")
    print("W&B UI에서 수동으로 'production' alias를 추가해 주세요.")

In [ ]:
wandb.finish()
print("모든 W&B 리소스가 정리되었습니다. 데모 완료!")